# Shape Tech-Test

Author: Jonatas Cesar 

In [ ]:
import matplotlib.pyplot as plt
import pandas
import pandas as pd
import shap
from catboost import CatBoostClassifier, Pool
from pandas_profiling import ProfileReport
from sklearn.metrics import RocCurveDisplay, classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight

In [ ]:
#%matplotlib inline

In [ ]:
df = pd.read_excel("O&G Equipment Data.xlsx", index_col="Cycle")

In [ ]:
df.describe().T

In [ ]:
# check imbalanced data
df.Fail.mean()

In [ ]:
profile = ProfileReport(df, title="Shape Data - Profiling Report", minimal=True)

In [ ]:
profile.to_widgets()

## Split Data

In [ ]:
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## Model Fit 

In [ ]:
model = CatBoostClassifier()

In [ ]:
cat_features = ["Preset_1", "Preset_2"]

In [ ]:
model.fit(
    X_train,
    y_train,
    cat_features=cat_features,
    verbose=False,
)

## Classification perfomance

In [ ]:
y_pred_prob = model.predict(X_test, prediction_type="Probability")[:, 1]

In [ ]:
RocCurveDisplay.from_predictions(y_test.values, y_pred_prob)
plt.show()

In [ ]:
y_pred = y_pred_prob > 0.5

In [ ]:
print(classification_report(y_test.values, y_pred))

## Feature importance 

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(Pool(X_train, y_train, cat_features=cat_features))
shap.summary_plot(shap_values, X_train)